#### Import Variants and Misconceptions

By using the math module, I would like to briefly discuss the various import variants such as:

* `import math`
* `from math import sqrt, abs`
* `from math import *`
* `import math as r_math`
* `from math import sqrt as r_sqrt`

##### import math

* loads the entire module (`math`) in memory if it's not already there
* adds a reference to it in `sys.modules` with a key of `math`
* adds a symbol of the same name (`math`) in our current namespace referencing the `math` object

##### import math as r_math

* loads the entire module (`math`) in memory if it's not already there
* adds a reference to it in `sys.modules` with a key of `math`
* adds the symbol `r_math` to our current namespace referencing the `math` object

##### from math import sqrt

* loads the entire module (`math`) in memory if it's not already there
* adds a reference to it in `sys.modules` with a key of `math`
* adds the symbol `sqrt` to our current namespace referencing the `math.sqrt` function
* it **does not** add the symbol `math` to our current namespace

##### from math import sqrt as r_sqrt

* loads the entire module (`math`) in memory if it's not already there
* adds a reference to it in `sys.modules` with a key of `math`
* adds the symbol `r_sqrt` to our current namespace referencing the `math.sqrt` function
* it **does not** add the symbol `math` to our current namespace

##### from math import *

* loads the entire module (`math`) in memory if it's not already there
* adds a reference to it in `sys.modules` with a key of `math`
* adds symbols for all exported symbols in the `math` module directly to our name space (we'll see how what is exported from a module/package can be controlled using underscores or `__all__` later)
* it **does not** add the symbol `math` to our current namespace

As you can see, in **every** instance, the module is imported and a reference to it is added to `sys.modules`. The variants really have to do with what is injected into our current **namespace**: the module name, an alias to it, just the specified symbols from the module, or all the exported symbols from the module.

#### Misconceptions

This leads to the first misconception:

"You should use

`from math import sqrt, abs`

rather than 

`import math`

because that way you only import what you need and you're not having Python load the entire module?"

At the end of the day, the module is always loaded and cached (`sys.modules`), these different variants of the `import` statement merely determine what symbols are added to our module (global) namespace. That's it and by the way you can use this tric to see what was inside your namespace at a certain point in time durring the runtime.



#### Efficiency

The final thing we need to look into is often mentioned in various blog posts and online discussions.

`import variant #1` is more "efficient" than `import variant #2`

Maybe so, but realistically by how much?

Or even how the following is terribly wrong because it re-imports the `math` module **every** time `my_func` is called:

In [39]:
def my_func(a):
    import math
    return math.sqrt(a)

From a readability standpoint, yes, that is **not** a good idea. Much better to put all your imports at the top of the module once in a location where any reader can easily see all your module dependencies.

But as far as reloading the module, you should now understand that's absolutely not true. Instead, it has to do a dictionary lookup in the `sys.modules` dictionary, not reload the entire module after the first load has occurred!

Dictionary lookups are blazingly fast in Python - so, yes, there is some overhead, but not as much as you may think.

So, let's write some timing code to test these things and see how they compare.

We shoudl consider both relative speed differences as well as absilute speed differences.

If you try to optimize your code and end up reducing that code's speed by 50% that sounds good. But what if the original code ran in `1`s. Now it runs in `0.5`s. How long does the total program run? Down from `30`s to `29.5`s? Things are relative...

In [40]:
from time import perf_counter

I'm also going to write a small utility function that compares two timings to each other:

In [41]:
from collections import namedtuple

Timings = namedtuple('Timings', 'timing_1 timing_2 abs_diff rel_diff_perc')
def compare_timings(timing1, timing2):
    rel_diff = (timing2 - timing1)/timing1 * 100
    
    timings = Timings(round(timing1, 1),
                     round(timing2, 1),
                     round(timing2 - timing1, 2),
                     round(rel_diff, 2))
    return timings

##### Timing using fully qualified `module.symbol` 

In [42]:
test_repeats = 10_000_000

In [43]:
import math

start = perf_counter()
for _ in range(test_repeats):
    math.sqrt(2)
end = perf_counter()
elapsed_fully_qualified = end - start
print(f'Elapsed: {elapsed_fully_qualified}')

Elapsed: 2.057656398357829


##### Timing using a directly imported symbol name:

In [44]:
from math import sqrt

start = perf_counter()
for _ in range(test_repeats):
    sqrt(2)
end = perf_counter()
elapsed_direct_symbol = end - start
print(f'Elapsed: {elapsed_direct_symbol}')

Elapsed: 1.603430354697538


Let's see the relative and absolute time differences:

In [45]:
compare_timings(elapsed_fully_qualified, elapsed_direct_symbol)

Timings(timing_1=2.1, timing_2=1.6, abs_diff=-0.45, rel_diff_perc=-22.07)

Definitely faster - but in absolute terms I really did not save a whole lot - over `10,000,000` iterations!

##### Timing using a function (fully qualified symbol)

In [46]:
import math

def func():
    math.sqrt(2)
    
start = perf_counter()
for _ in range(test_repeats):
    func()
end = perf_counter()
elapsed_func_fully_qualified = end - start
print(f'Elapsed: {elapsed_func_fully_qualified}') 

Elapsed: 3.2668947610088703


In [47]:
compare_timings(elapsed_fully_qualified, elapsed_func_fully_qualified)

Timings(timing_1=2.1, timing_2=3.3, abs_diff=1.21, rel_diff_perc=58.77)

That was slower because of the function call overhead, but not by much in absolute terms considering I called `func()` `10,000,000` times!

##### Timing using a function (direct symbol)

In [48]:
from math import sqrt

def func():
    sqrt(2)
    
start = perf_counter()
for _ in range(test_repeats):
    func()
end = perf_counter()
elapsed_func_direct_symbol = end - start
print(f'Elapsed: {elapsed_func_direct_symbol}')

Elapsed: 2.80123663975316


In [49]:
compare_timings(elapsed_func_fully_qualified, elapsed_func_direct_symbol)

Timings(timing_1=3.3, timing_2=2.8, abs_diff=-0.47, rel_diff_perc=-14.25)

Slower, but again not by much in absolute terms considering this was for `10,000,000` iterations.

##### Timing using a nested import (fully qualified symbol)

In [50]:
def func():
    import math
    math.sqrt(2)
    
start = perf_counter()
for _ in range(test_repeats):
    func()
end = perf_counter()
elapsed_nested_fully_qualified = end - start
print(f'Elapsed: {elapsed_nested_fully_qualified}')

Elapsed: 5.041648347331877


In [51]:
compare_timings(elapsed_func_fully_qualified, elapsed_nested_fully_qualified)

Timings(timing_1=3.3, timing_2=5.0, abs_diff=1.77, rel_diff_perc=54.33)

So definitely slower. But in absolute terms, for `10,000,000` iterations?

##### Timing using a nested import (direct symbol)

In [52]:
def func():
    from math import sqrt
    sqrt(2)
    
start = perf_counter()
for _ in range(test_repeats):
    func()
end = perf_counter()
elapsed_nested_direct_symbol = end - start
print(f'Elapsed: {elapsed_nested_direct_symbol}')

Elapsed: 14.60262281403945


In [53]:
compare_timings(elapsed_nested_fully_qualified, elapsed_nested_direct_symbol)

Timings(timing_1=5.0, timing_2=14.6, abs_diff=9.56, rel_diff_perc=189.64)

That was significantly slower! Even in absolute terms.

So does this mean you should put imports inside functions?

No, of course not - follow the convention, it makes code far more readable, and of course optimize your code only once you have identified the bottlenecks. 

Does this mean you shouldn't care at all about the performance of your code based on the import variants?

Again, of course not - you absolutely should.

But, there is absolutely no reason to re-write your code from 

`import math
math.sqrt(2)`

to 

`from math import sqrt
sqrt(2)
`

for **speed** reasons if during the entire lifetime of your application you only call that function `100` times... or `10,000,000` times.

Really depends on your circumstance - be aware of it, but don't try to optimize code until you know **where** you **need** to optimize!



And

`from module import *`

It's not evil, just not very safe - again depends on your circumstance. Python community strongly advised to not use that for readability purposes also.